<a href="https://colab.research.google.com/github/ayushksingh28/torch_itt/blob/main/torch_cnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install torch
import torch

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [12]:
class NN(nn.Module):
  def __init__(self, input_size, num_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50)
    self.fc2 = nn.Linear(50, num_classes)

  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x

In [20]:
class CNN(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (3,3), stride = (1, 1), padding = (1, 1))
    self.pool = nn.MaxPool2d(kernel_size = (2,2), stride = (2, 2))
    self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (3, 3), stride = (1, 1), padding = (1, 1))
    self.fc1 = nn.Linear(16*7*7, num_classes)


  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool(x)
    x = F.relu(self.conv2(x))
    x = self.pool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc1(x)

    return x


In [21]:
in_channels = 1
num_classes = 10
learning_rate = 0.001
num_epochs = 3
batch_size = 64

In [22]:
train_dataset = datasets.MNIST(root = "datasets/", train = True, transform = transforms.ToTensor(), download = True)

100%|██████████| 9912422/9912422 [00:00<00:00, 110887488.12it/s]


Extracting datasets/MNIST/raw/train-images-idx3-ubyte.gz to datasets/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 82573751.75it/s]


Extracting datasets/MNIST/raw/train-labels-idx1-ubyte.gz to datasets/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 28596615.97it/s]


Extracting datasets/MNIST/raw/t10k-images-idx3-ubyte.gz to datasets/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 9759492.20it/s]

Extracting datasets/MNIST/raw/t10k-labels-idx1-ubyte.gz to datasets/MNIST/raw



In [23]:
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle =True)

In [24]:
test_dataset = datasets.MNIST(root = "datasets/", train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle =True)

In [25]:
model =CNN().to(device)

In [26]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [28]:
for epoch in range(num_epochs):
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)

        scores = model(data)
        loss = criterion(scores, targets)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

In [29]:
def check_accuracy(loader, model):
  num_correct = 0
  num_samples = 0
  model.eval()

  with torch.no_grad():
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)

      scores = model(x)
      _, predictions = scores.max(1)
      num_correct += (predictions == y).sum()
      num_samples += predictions.size(0)

    print(f'got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)')
  model.train()
  return num_correct / num_samples

In [30]:
print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")

got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)
Accuracy on training set: 98.24
got (num_correct)/(num_samples) with accuracy (float(num_correct)/float(num_samples)*100:.2f)
Accuracy on test set: 98.19
